# citation for noise methods: https://github.com/hendrycks/robustness/blob/master/ImageNet-C/create_c/make_cifar_c.py

https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv

In [1]:
import cv2 
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
train_x = torch.load('train_x.pt', map_location=torch.device('cpu'))
train_y = torch.load('train_y.pt', map_location=torch.device('cpu'))
test_x = torch.load('test_x.pt', map_location=torch.device('cpu'))
test_y = torch.load('test_y.pt', map_location=torch.device('cpu'))

In [3]:
print("train_x shape: ", train_x.size())
print("train_y shape: ", train_y.size())
print("test_x shape: ", test_x.size())
print("test_y shape: ", test_y.size())

train_x shape:  torch.Size([39209, 50, 50, 3])
train_y shape:  torch.Size([39209])
test_x shape:  torch.Size([12630, 50, 50, 3])
test_y shape:  torch.Size([12630])


In [4]:
train_x = train_x.numpy().astype('float32')
train_y = train_y.numpy().astype('float32')
test_x = test_x.numpy().astype('float32')
test_y = test_y.numpy().astype('float32')

In [5]:
print("train_x shape: ", train_x.shape)
print("train_y shape: ", train_y.shape)
print("test_x shape: ", test_x.shape)
print("test_y shape: ", test_y.shape)

train_x shape:  (39209, 50, 50, 3)
train_y shape:  (39209,)
test_x shape:  (12630, 50, 50, 3)
test_y shape:  (12630,)


In [6]:
idx = np.random.permutation(train_x.shape[0])
train_x_shuffle, train_y_shuffle = train_x[idx], train_y[idx]
idx = np.random.permutation(test_x.shape[0])
test_x_shuffle, test_y_shuffle = test_x[idx], test_y[idx]

In [7]:
train_x_shuffle = np.swapaxes(train_x_shuffle,3,1)
test_x_shuffle = np.swapaxes(test_x_shuffle,3,1)
train_x_shuffle = np.swapaxes(train_x_shuffle,2,3)
test_x_shuffle = np.swapaxes(test_x_shuffle,2,3)

In [8]:
val_size = int(train_x.shape[0] * 0.2)
val_x = train_x_shuffle[0:val_size]
val_y = train_y_shuffle[0:val_size]
train_x_shuffle = train_x_shuffle[val_size:]
train_y_shuffle = train_y_shuffle[val_size:]

In [9]:
train_x = torch.from_numpy(train_x_shuffle)
train_y = torch.from_numpy(train_y_shuffle)
val_x = torch.from_numpy(val_x)
val_y = torch.from_numpy(val_y)
test_x = torch.from_numpy(test_x_shuffle)
test_y = torch.from_numpy(test_y_shuffle)

In [10]:
train_x = train_x.numpy().astype('float32')
train_y = train_y.numpy().astype('float32')
val_x = val_x.numpy().astype('float32')
val_y = val_y.numpy().astype('float32')
test_x = test_x.numpy().astype('float32')
test_y = test_y.numpy().astype('float32')

In [11]:
print("train_x shape: ", train_x.shape)
print("train_y shape: ", train_y.shape)
print("val_x shape: ", val_x.shape)
print("val_y shape: ", val_y.shape)
print("test_x shape: ", test_x.shape)
print("test_y shape: ", test_y.shape)

train_x shape:  (31368, 3, 50, 50)
train_y shape:  (31368,)
val_x shape:  (7841, 3, 50, 50)
val_y shape:  (7841,)
test_x shape:  (12630, 3, 50, 50)
test_y shape:  (12630,)


In [12]:
train_x_rotate = np.zeros((train_x.shape[0] * 4, train_x.shape[1], train_x.shape[2], train_x.shape[3]))
train_y_rotate = np.zeros((train_y.shape[0] * 4, 2))
val_x_rotate = np.zeros((val_x.shape[0] * 4, val_x.shape[1], val_x.shape[2], val_x.shape[3]))
val_y_rotate = np.zeros((val_y.shape[0] * 4, 2))
test_x_rotate = np.zeros((test_x.shape[0] * 4, test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y_rotate = np.zeros((test_y.shape[0] * 4, 2))

In [13]:
print("train_x_rotate shape: ", train_x_rotate.shape)
print("train_y_rotate shape: ", train_y_rotate.shape)
print("val_x_rotate shape: ", val_x_rotate.shape)
print("val_y_rotate shape: ", val_y_rotate.shape)
print("test_x_rotate shape: ", test_x_rotate.shape)
print("test_y_rotate shape: ", test_y_rotate.shape)

train_x_rotate shape:  (125472, 3, 50, 50)
train_y_rotate shape:  (125472, 2)
val_x_rotate shape:  (31364, 3, 50, 50)
val_y_rotate shape:  (31364, 2)
test_x_rotate shape:  (50520, 3, 50, 50)
test_y_rotate shape:  (50520, 2)


In [14]:
for i in range(train_x.shape[0]):
    train_x_rotate[i * 4 + 0] = train_x[i]
    train_y_rotate[i * 4 + 0][0] = train_y[i]
    train_y_rotate[i * 4 + 0][1] = 0
    
    train_x_rotate[i * 4 + 1][0] = np.rot90(train_x[i][0])
    train_x_rotate[i * 4 + 1][1] = np.rot90(train_x[i][1])
    train_x_rotate[i * 4 + 1][2] = np.rot90(train_x[i][2])
    train_y_rotate[i * 4 + 1][0] = train_y[i]
    train_y_rotate[i * 4 + 1][1] = 1
    
    train_x_rotate[i * 4 + 2][0] = np.rot90(np.rot90(train_x[i][0]))
    train_x_rotate[i * 4 + 2][1] = np.rot90(np.rot90(train_x[i][1]))
    train_x_rotate[i * 4 + 2][2] = np.rot90(np.rot90(train_x[i][2]))
    train_y_rotate[i * 4 + 2][0] = train_y[i]
    train_y_rotate[i * 4 + 2][1] = 2
    
    train_x_rotate[i * 4 + 3][0] = np.rot90(np.rot90(np.rot90(train_x[i][0])))
    train_x_rotate[i * 4 + 3][1] = np.rot90(np.rot90(np.rot90(train_x[i][1])))
    train_x_rotate[i * 4 + 3][2] = np.rot90(np.rot90(np.rot90(train_x[i][2])))
    train_y_rotate[i * 4 + 3][0] = train_y[i]
    train_y_rotate[i * 4 + 3][1] = 3
    
for i in range(val_x.shape[0]):
    val_x_rotate[i * 4 + 0] = val_x[i]
    val_y_rotate[i * 4 + 0][0] = val_y[i]
    val_y_rotate[i * 4 + 0][1] = 0
    
    val_x_rotate[i * 4 + 1][0] = np.rot90(val_x[i][0])
    val_x_rotate[i * 4 + 1][1] = np.rot90(val_x[i][1])
    val_x_rotate[i * 4 + 1][2] = np.rot90(val_x[i][2])
    val_y_rotate[i * 4 + 1][0] = val_y[i]
    val_y_rotate[i * 4 + 1][1] = 1
    
    val_x_rotate[i * 4 + 2][0] = np.rot90(np.rot90(val_x[i][0]))
    val_x_rotate[i * 4 + 2][1] = np.rot90(np.rot90(val_x[i][1]))
    val_x_rotate[i * 4 + 2][2] = np.rot90(np.rot90(val_x[i][2]))
    val_y_rotate[i * 4 + 2][0] = val_y[i]
    val_y_rotate[i * 4 + 2][1] = 2
    
    val_x_rotate[i * 4 + 3][0] = np.rot90(np.rot90(np.rot90(val_x[i][0])))
    val_x_rotate[i * 4 + 3][1] = np.rot90(np.rot90(np.rot90(val_x[i][1])))
    val_x_rotate[i * 4 + 3][2] = np.rot90(np.rot90(np.rot90(val_x[i][2])))
    val_y_rotate[i * 4 + 3][0] = val_y[i]
    val_y_rotate[i * 4 + 3][1] = 3
    
for i in range(test_x.shape[0]):
    test_x_rotate[i * 4 + 0] = test_x[i]
    test_y_rotate[i * 4 + 0][0] = test_y[i]
    test_y_rotate[i * 4 + 0][1] = 0
    
    test_x_rotate[i * 4 + 1][0] = np.rot90(test_x[i][0])
    test_x_rotate[i * 4 + 1][1] = np.rot90(test_x[i][1])
    test_x_rotate[i * 4 + 1][2] = np.rot90(test_x[i][2])
    test_y_rotate[i * 4 + 1][0] = test_y[i]
    test_y_rotate[i * 4 + 1][1] = 1
    
    test_x_rotate[i * 4 + 2][0] = np.rot90(np.rot90(test_x[i][0]))
    test_x_rotate[i * 4 + 2][1] = np.rot90(np.rot90(test_x[i][1]))
    test_x_rotate[i * 4 + 2][2] = np.rot90(np.rot90(test_x[i][2]))
    test_y_rotate[i * 4 + 2][0] = test_y[i]
    test_y_rotate[i * 4 + 2][1] = 2
    
    test_x_rotate[i * 4 + 3][0] = np.rot90(np.rot90(np.rot90(test_x[i][0])))
    test_x_rotate[i * 4 + 3][1] = np.rot90(np.rot90(np.rot90(test_x[i][1])))
    test_x_rotate[i * 4 + 3][2] = np.rot90(np.rot90(np.rot90(test_x[i][2])))
    test_y_rotate[i * 4 + 3][0] = test_y[i]
    test_y_rotate[i * 4 + 3][1] = 3

In [16]:
print("test_x shape: ", test_x.shape)
print("test_x_rotate shape: ", test_x_rotate.shape)

test_x shape:  (12630, 3, 50, 50)
test_x_rotate shape:  (50520, 3, 50, 50)


In [17]:
test_x = np.swapaxes(test_x,3,1)
test_x_rotate = np.swapaxes(test_x_rotate,3,1)
test_x = np.swapaxes(test_x,2,1)
test_x_rotate = np.swapaxes(test_x_rotate,2,1)
print("test_x shape: ", test_x.shape)
print("test_x_rotate shape: ", test_x_rotate.shape)

test_x shape:  (12630, 50, 50, 3)
test_x_rotate shape:  (50520, 50, 50, 3)


In [52]:
test_y = torch.from_numpy(test_y)
test_y_rotate = torch.from_numpy(test_y_rotate)
print(test_y.size())
print(test_y_rotate.size())

torch.Size([12630])
torch.Size([50520, 2])


# prepare gauss noise

In [24]:
def gauss_noisy(image):
  row,col,ch= image.shape
  mean = 0
  var = 0.1
  sigma = var**0.5
  gauss = np.random.normal(mean,sigma,(row,col,ch))
  gauss = gauss.reshape(row,col,ch)
  noisy = image + gauss
  return noisy

In [30]:
test_x_gauss_noise = np.zeros(test_x.shape)
test_x_rotate_gauss_noise = np.zeros(test_x_rotate.shape)
print("test_x_gauss_noise shape: ", test_x_gauss_noise.shape)
print("test_x_rotate_gauss_noise shape: ", test_x_rotate_gauss_noise.shape)

test_x_gauss_noise shape:  (12630, 50, 50, 3)
test_x_rotate_gauss_noise shape:  (50520, 50, 50, 3)


In [31]:
for i in range(test_x.shape[0]):
    test_x_gauss_noise[i] = gauss_noisy(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_gauss_noise[i] = gauss_noisy(test_x_rotate[i])

In [34]:
test_x_gauss_noise = np.swapaxes(test_x_gauss_noise,3,1)
test_x_rotate_gauss_noise = np.swapaxes(test_x_rotate_gauss_noise,3,1)
test_x_gauss_noise = np.swapaxes(test_x_gauss_noise,2,3)
test_x_rotate_gauss_noise = np.swapaxes(test_x_rotate_gauss_noise,2,3)
print("test_x_gauss_noise shape: ", test_x_gauss_noise.shape)
print("test_x_rotate_gauss_noise shape: ", test_x_rotate_gauss_noise.shape)

test_x_gauss_noise shape:  (12630, 3, 50, 50)
test_x_rotate_gauss_noise shape:  (50520, 3, 50, 50)


In [35]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_gauss_noise[:,i,:,:] = (test_x_gauss_noise[:,i,:,:] - m) / s
    test_x_rotate_gauss_noise[:,i,:,:] = (test_x_rotate_gauss_noise[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.44989, 79.546646, 82.43356]
Train std: [70.71926, 67.62619, 69.54247]


In [36]:
test_x_gauss_noise = torch.from_numpy(test_x_gauss_noise)
test_x_rotate_gauss_noise = torch.from_numpy(test_x_rotate_gauss_noise)
print(test_x_gauss_noise.size())
print(test_x_rotate_gauss_noise.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [37]:
torch.save(test_x_gauss_noise, 'test_x_gauss_noise.pt')
torch.save(test_y, 'test_y_gauss_noise.pt')
torch.save(test_x_rotate_gauss_noise, 'test_x_rotate_gauss_noise.pt')
torch.save(test_y_rotate, 'test_y_rotate_gauss_noise.pt')

# prepare defocus blur

In [41]:
def disk(radius, alias_blur=0.1, dtype=np.float32):
    if radius <= 8:
        L = np.arange(-8, 8 + 1)
        ksize = (3, 3)
    else:
        L = np.arange(-radius, radius + 1)
        ksize = (5, 5)
    X, Y = np.meshgrid(L, L)
    aliased_disk = np.array((X ** 2 + Y ** 2) <= radius ** 2, dtype=dtype)
    aliased_disk /= np.sum(aliased_disk)

    # supersample disk to antialias
    return cv2.GaussianBlur(aliased_disk, ksize=ksize, sigmaX=alias_blur)

def defocus_blur(x, severity=1):
    c = [(0.3, 0.4), (0.4, 0.5), (0.5, 0.6), (1, 0.2), (1.5, 0.1)][severity - 1]

    x = np.array(x) / 255.
    kernel = disk(radius=c[0], alias_blur=c[1])

    channels = []
    for d in range(3):
        channels.append(cv2.filter2D(x[:, :, d], -1, kernel))
    channels = np.array(channels).transpose((1, 2, 0))  # 3x32x32 -> 32x32x3

    return np.clip(channels, 0, 1) * 255

In [39]:
test_x_defocus_blur = np.zeros(test_x.shape)
test_x_rotate_defocus_blur = np.zeros(test_x_rotate.shape)
print("test_x_defocus_blur shape: ", test_x_defocus_blur.shape)
print("test_x_rotate_defocus_blur shape: ", test_x_rotate_defocus_blur.shape)

test_x_defocus_blur shape:  (12630, 50, 50, 3)
test_x_rotate_defocus_blur shape:  (50520, 50, 50, 3)


In [43]:
for i in range(test_x.shape[0]):
    test_x_defocus_blur[i] = defocus_blur(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_defocus_blur[i] = defocus_blur(test_x_rotate[i])

In [53]:
test_x_defocus_blur = np.swapaxes(test_x_defocus_blur,3,1)
test_x_rotate_defocus_blur = np.swapaxes(test_x_rotate_defocus_blur,3,1)
test_x_defocus_blur = np.swapaxes(test_x_defocus_blur,2,3)
test_x_rotate_defocus_blur = np.swapaxes(test_x_rotate_defocus_blur,2,3)
print("test_x_defocus_blur shape: ", test_x_defocus_blur.shape)
print("test_x_rotate_defocus_blur shape: ", test_x_rotate_defocus_blur.shape)

test_x_defocus_blur shape:  (12630, 3, 50, 50)
test_x_rotate_defocus_blur shape:  (50520, 3, 50, 50)


In [54]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_defocus_blur[:,i,:,:] = (test_x_defocus_blur[:,i,:,:] - m) / s
    test_x_rotate_defocus_blur[:,i,:,:] = (test_x_rotate_defocus_blur[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.44989, 79.546646, 82.43356]
Train std: [70.71926, 67.62619, 69.54247]


In [55]:
test_x_defocus_blur = torch.from_numpy(test_x_defocus_blur)
test_x_rotate_defocus_blur = torch.from_numpy(test_x_rotate_defocus_blur)
print(test_x_defocus_blur.size())
print(test_x_rotate_defocus_blur.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [56]:
torch.save(test_x_defocus_blur, 'test_x_defocus_blur.pt')
torch.save(test_y, 'test_y_defocus_blur.pt')
torch.save(test_x_rotate_defocus_blur, 'test_x_rotate_defocus_blur.pt')
torch.save(test_y_rotate, 'test_y_rotate_defocus_blur.pt')